In [ ]:
import os, sqlite3
import numpy as np
import pandas as pd
from tqdm import tqdm

import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
dbs_to_index = os.listdir('/home/Shared/cultivar_sim_exps')
dbs_to_index = [e for e in dbs_to_index if e.split('.')[-1] == 'sqlite']

In [ ]:
def get_sql_tables(
        sql_path = f"/home/Shared/cultivar_sim_exps/{'sim_1698728407_78912.sqlite'}",
        table_list = ['Ids'] #'DefaultCultivarsAll', 'Genotypes', 'Results'
        ):
    db = sqlite3.connect(sql_path)
    return [pd.read_sql_query(f"SELECT * from {table_name}", db) for table_name in table_list]

In [ ]:
print('Updating shared tables')
# expect this to be the same table over and over
DefaultCultivarsAll = [get_sql_tables(
        sql_path = f"/home/Shared/cultivar_sim_exps/{db_name}",
        table_list = ['DefaultCultivarsAll']
        )[0].assign(File = db_name) for db_name in dbs_to_index]

DefaultCultivarsAll = pd.concat(DefaultCultivarsAll)

DefaultCultivarsAll = DefaultCultivarsAll.drop(columns = 'File').drop_duplicates()
DefaultCultivarsAll.shape

Updating shared tables


(58, 48)

In [ ]:
expect_dupe = []
expect_uniq = []

for db_name in dbs_to_index:
        db = sqlite3.connect(f"/home/Shared/cultivar_sim_exps/{db_name}")
        if expect_dupe == []:
            dupe = pd.read_sql_query(f"SELECT * FROM Genotypes WHERE Genotype NOT LIKE 'Cultivar%'", db)
            expect_dupe.append(dupe.assign(File = db_name) )
        expect_uniq.append(pd.read_sql_query(f"SELECT * FROM Genotypes WHERE Genotype     LIKE 'Cultivar%'", db).assign(File = db_name))

Genotypes = pd.concat(expect_dupe+expect_uniq)
Genotypes.File = Genotypes.File.str.replace('.sqlite', '')
# Reorder cols
Genotypes = Genotypes.loc[:, ['File', 'Genotype']+[e for e in list(Genotypes) if e not in ['File', 'Genotype']]]
del expect_dupe
del expect_uniq
Genotypes.shape

(3203, 17)

In [ ]:
Ids = [get_sql_tables(
        sql_path = f"/home/Shared/cultivar_sim_exps/{db_name}",
        table_list = ['Ids']
        )[0].assign(File = db_name) for db_name in dbs_to_index]

Ids = pd.concat(Ids)
Ids.SoilIdx = Ids.SoilIdx.astype(int)
Ids.File = Ids.File.str.replace('.sqlite', '')
# Reorder cols
Ids = Ids.loc[:, ['File']+[e for e in list(Ids) if e not in ['File']]]
Ids.shape

(2837275, 7)

In [ ]:
# DefaultCultivarsAll.head()

In [ ]:
# Genotypes.head()

In [ ]:
# Ids.head()

In [ ]:
# save out global referenc tables as parquet files. 
# check if files exist if so load and extend. 
def update_parquet_table(
        base_path = "/home/Shared/cultivar_sim_exps/",
        name = 'DefaultCultivarsAll',
        table = DefaultCultivarsAll
        ):
    file_path = f"{base_path}{name}.parquet"
    if os.path.exists(file_path):
        old_table = pq.read_table(file_path).to_pandas()
        table = old_table.merge(table)
    table = pa.Table.from_pandas(table)
    pq.write_table(table, file_path)

for k, v in zip(['DefaultCultivarsAll', 'Genotypes', 'Ids'], 
                [ DefaultCultivarsAll,   Genotypes,   Ids]):
    update_parquet_table(
        base_path = "/home/Shared/cultivar_sim_exps/",
        name =  k,
        table = v
        )

In [ ]:
# now for each of the files with results, convert and delete original sqlite

def results_sqlite_to_parquet(file_path = f"/home/Shared/cultivar_sim_exps/{'sim_1698728407_78912.sqlite'}",
                              delete_after = False,
                              double_dare = False # ignore file size requirement
                              ):
    # cut off sqlite, add parquet
    new_filepath = file_path[0:-6]+'parquet'

    table = get_sql_tables(sql_path = file_path, table_list = ['Results'] )[0]
    table.Date = table.Date.astype(int)
    table = pa.Table.from_pandas(table)
    pq.write_table(table, new_filepath)

    if delete_after:
        if os.path.exists(new_filepath):
            if os.path.getsize(new_filepath) > 1000000000:
                del_bool = True
            elif double_dare == True:
                del_bool = True
        if del_bool:
            os.remove(file_path)


In [ ]:
# sort from smallest to largest
tmp = {os.path.getsize(f"/home/Shared/cultivar_sim_exps/{db}"): db for db in dbs_to_index}
tmp = [tmp[e] for e in sorted( list(tmp.keys()) )]


In [ ]:
# print('staring conversion process')
# for db in tmp:
#     gb = round(os.path.getsize(f"/home/Shared/cultivar_sim_exps/{db}")/1000000000, 2)
#     print(f"{gb}G\t| {db}")
#     results_sqlite_to_parquet(file_path = f"/home/Shared/cultivar_sim_exps/{db}",
#                             delete_after = False,
#                             double_dare = False # ignore file size requirement
#                             )


In [ ]:
# pq.read_table(f"/home/Shared/cultivar_sim_exps/{'sim_cultivar.parquet'}").to_pandas()


In [ ]:
# pq.read_table(f"/home/Shared/cultivar_sim_exps/{'sim_1697187607_79518.parquet'}").to_pandas()


In [ ]:
# pq.read_table(f"/home/Shared/cultivar_sim_exps/{'sim_1698310807_76163.parquet'}").to_pandas()


In [ ]:
# len(dbs_to_index)
# for db in dbs_to_index[0:1]:
